In [29]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target #rotulos

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

w = modelo.coef_[0] # pesos do modelo treinado
Vs = X_test.iloc[0].to_dict() # utilizxando a primeira instância dos dados de teste 

# Inicializa a lista para armazenar os deltas
# o delta c\ peso negativo é (valor feature na instancia - valor min da feature no dataset) * o peso da feature 
# o delta c\ peso positivo é (vvalor max da feature no dataset - valor feature na instancia - ) * o peso da feature
delta = []


# Calcula o valor delta para cada feature
# peso negativo, valor menor = maior possibilidade da instância peetencer a classe
for i, feature in enumerate(df.columns[:-1]):  
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].min()) * w[i])
    else:
        delta.append((df[feature].max() - Vs[feature]) * w[i])

# Calcula o limiar (-Gamma_w)
R = abs(-sum(delta))  # Usando a equação 13 do artigo e garantindo que R seja positivo

print(f"Pesos: {w}")
print(f"Delta: {delta}")
print(f"Limiar R: {R:.2f}")
print(Vs)

Pesos: [-0.39381403  0.96220565 -2.37519275 -0.99872731]
Delta: [-0.7088652578943643, 1.539529034488898, -8.788213178604362, -1.098600036798368]
Limiar R: 9.06
{'sepal length (cm)': 6.1, 'sepal width (cm)': 2.8, 'petal length (cm)': 4.7, 'petal width (cm)': 1.2}


In [30]:
def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    delta_sorted = sorted(delta, reverse=True) # Ordena os valores delta em ordem decrescente
    R_atual = R # Inicializa o limiar atual
    Idx = 0 # Itera sobre os valores delta ordenados
   
    while R_atual >= 0:
        feature = X_test.columns[Idx] # nome da feature correspondente ao índice
        Xpl.append(feature) # Adiciona a feature à PI-explicação
        R_atual -= delta_sorted[Idx] # Atualiza o limiar atual
        Idx += 1 # Incrementa o índice
    
    return Xpl # Retorna a PI-explicação

In [31]:
# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

# Imprime a PI-explicação
print(f"PI-Explicação: {Xpl}")

IndexError: index 4 is out of bounds for axis 0 with size 4

In [41]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target  # Rótulos

# Divide o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina um modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Obtém os pesos do modelo
w = modelo.coef_[0]  # Pesos do modelo treinado

def one_explanation(Vs, w, df, R):
    """
    Função que computa uma única PI-explicação para um classificador linear,
    seguindo a lógica do algoritmo do artigo.

    Args:
        Vs: Valores da instância que está sendo explicada (como um dicionário).
        w: Vetor de pesos para cada feature (como um dicionário).
        df: DataFrame com os dados.
        R: Limiar para a explicação (-Gamma_w).

    Returns:
        Uma lista de features que representa a PI-explicação.
    """

    # Inicializa a lista de PI-explicação
    Xpl = []

    # Inicializa o limiar atual
    R_atual = R

    # Calcula os deltas para cada feature
    delta = []
    for i, feature in enumerate(df.columns[:-1]):
        if w[i] < 0:
            delta.append((Vs[feature] - df[feature].min()) * w[i])
        else:
            delta.append((df[feature].max() - Vs[feature]) * w[i])

    # Ordena os valores delta em ordem decrescente
    delta_sorted = sorted(delta, reverse=True)

    # Calcula o limiar (-Gamma_w) -  Ajustado para garantir que R seja menor que a soma dos deltas
    R_inicial = abs(-sum(delta))  # Usando a equação 13 do artigo e garantindo que R seja positivo

    # Itera sobre os valores delta ordenados
    Idx = 0
    R_atual = R_inicial  # R_atual é inicializado com o limiar R_inicial
    while R_atual >= 0:  # Ajusta a condição do loop
        # Incrementa o índice
        Idx += 1
        # Atualiza o limiar atual
        R_atual -= delta_sorted[Idx]
        # Obtém o nome da feature correspondente ao índice
        feature = X_test.columns[Idx]

        # Adiciona a feature à PI-explicação
        Xpl.append(feature)



    # Retorna a PI-explicação
    return Xpl

# Seleciona uma instância do dataset de teste
Vs = X_test.iloc[0].to_dict()  # Utilizando a primeira instância dos dados de teste 

# Calcula o limiar (-Gamma_w) -  Ajustado para garantir que R seja menor que a soma dos deltas
R = abs(-sum(delta)) # Usando a equação 13 do artigo e garantindo que R seja positivo

# Computa a PI-explicação
Xpl = one_explanation(Vs, w, df, R)

# Imprime a PI-explicação
print(f"Pesos: {w}")
print(f"Delta: {delta}")
print(f"Limiar R: {R:.2f}")
print(Vs)
print(f"PI-Explicação: {Xpl}")

IndexError: list index out of range

In [ ]:
def one_explanation(Vs, delta, R):
    Xpl = [] # é a lista de pi-explicação
    delta_sorted = sorted(delta, reverse=True) # Ordena os valores delta em ordem decrescente
    R_atual = R # Inicializa o limiar atual
    Idx = 0 # Itera sobre os valores delta ordenados
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        # Obtém o nome da feature correspondente ao índice
        feature = X_test.columns[Idx]

        # Adiciona a feature à PI-explicação
        Xpl.append(feature)

        # Atualiza o limiar atual
        R_atual -= delta_sorted[Idx]

        # Incrementa o índice
        Idx += 1

    # Retorna a PI-explicação
    return Xpl

In [ ]:
def one_explanation(Vs, delta, R):

 
 Xpl = [] # é a lista de pi-explicação

    # Ordena os valores delta em ordem decrescente
    delta_sorted = sorted(delta, reverse=True)

    # Inicializa o limiar atual
    R_atual = R

    # Itera sobre os valores delta ordenados
    Idx = 0
    while R_atual >= 0 and Idx < len(delta_sorted):
        # Obtém o nome da feature correspondente ao índice
        feature = X_test.columns[Idx]

        # Adiciona a feature à PI-explicação
        Xpl.append(feature)

        # Atualiza o limiar atual
        R_atual -= delta_sorted[Idx]

        # Incrementa o índice
        Idx += 1

    # Retorna a PI-explicação
    return Xpl

In [17]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def one_explanation(Vs, delta, R):
    """
    Função que computa uma única PI-explicação para um classificador linear,
    seguindo a lógica do algoritmo do artigo.

    Args:
        Vs: Valores da instância que está sendo explicada (como um dicionário).
        delta: Lista de valores delta para cada feature.
        R: Limiar para a explicação (-Gamma_w).

    Returns:
        Uma lista de features que representa a PI-explicação.
    """

    Xpl = [] # é a lista de pi-explicação

    # Ordena os valores delta em ordem decrescente
    delta_sorted = sorted(delta, reverse=True)

    # Inicializa o limiar atual
    R_atual = R

    # Itera sobre os valores delta ordenados
    Idx = 0
    while R_atual >= 0 and Idx < len(delta_sorted):
        # Obtém o nome da feature correspondente ao índice
        feature = X_test.columns[Idx]

        # Adiciona a feature à PI-explicação
        Xpl.append(feature)

        # Atualiza o limiar atual
        R_atual -= delta_sorted[Idx]

        # Incrementa o índice
        Idx += 1

    # Retorna a PI-explicação
    return Xpl

def compute_delta(df, w, Vs):
    """
    Função que computa o valor delta para cada feature, seguindo as regras do artigo.

    Args:
        df: DataFrame com os dados.
        w: Vetor de pesos para cada feature (como um dicionário).
        Vs: Valores da instância que está sendo explicada (como um dicionário).

    Returns:
        Uma lista de valores delta para cada feature.
    """

    delta = []
    for feature in df.columns:
        # Se o peso for negativo
        if w[feature] < 0:
            # Calcula o valor delta como (valor da instância - valor mínimo da feature) * peso da feature
            delta.append((Vs[feature] - df[feature].min()) * w[feature])
        else:
            # Calcula o valor delta como (valor máximo da feature - valor da instância) * peso da feature
            delta.append((df[feature].max() - Vs[feature]) * w[feature])

    return delta

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Divide o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2)

# Armazena os nomes das colunas originais
colunas_originais = X_train.columns

# Escalona os dados de treino e teste usando StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converte X_train e X_test de volta para DataFrames pandas
X_train = pd.DataFrame(X_train, columns=colunas_originais)
X_test = pd.DataFrame(X_test, columns=colunas_originais)

# Treina um modelo de regressão logística
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Obtém os pesos do modelo
w = dict(zip(X_train.columns, model.coef_[0]))

# Seleciona uma instância do dataset de teste
Vs = X_test.iloc[0].to_dict()

# Calcula o valor delta para cada feature
delta = compute_delta(df.drop('target', axis=1), w, Vs)

# Calcula o limiar (-Gamma_w)
R = -sum(delta)  # Usando a equação 13 do artigo

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

# Imprime a PI-explicação
print(Xpl)

[]
